In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
!pip install -q transformers bitsandbytes datasets peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.0 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Sun Aug 24 14:47:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import json
import math
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig, EarlyStoppingCallback
from typing import List, Dict, Optional
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from torch.utils.data import DataLoader

In [13]:
class LoRATrainer:
    def __init__(self, data_path: str, model_name: str):
        self.data_path = data_path
        self.model_name = model_name

        self.lora_config = None
        self.bnb_config = None
        self.model = None

        self.tokenizer = None
        self.train_tokenized_data = None
        self.validation_tokenized_data = None
        self.test_tokenized_data = None

        self.train_dataset = None
        self.validation_dataset = None
        self.test_dataset = None

        if self.model_name is None or not self.model_name.strip():
            raise ValueError("Model name cannot be none or empty")

    def set_lora_conf(self, lora_config: Optional[Dict] = None):
        if lora_config is None or not lora_config:
            lora_config = {
                "r":8,
                "lora_alpha":16,
                "target_modules":['q_proj', 'v_proj'],
                "lora_dropout":0.05,
                "bias":"none",
                "task_type": TaskType.CAUSAL_LM
            }

        self.lora_config = LoraConfig(**lora_config)
        return self.lora_config

    def set_bnb_config(self, bnb_config: Optional[Dict] = None):
        if not bnb_config or bnb_config is None:
            bnb_config = {
                "load_in_4bit":True,
                "bnb_4bit_quant_type":'nf4',
                "bnb_4bit_compute_dtype":torch.bfloat16
            }

        self.bnb_config = BitsAndBytesConfig(**bnb_config)
        return self.bnb_config

    def load_model(self):
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config = self.bnb_config,
            device_map = "auto",
            trust_remote_code = True
        )
        self.model = get_peft_model(self.model, self.lora_config)
        return self.model

    def preprocess_data(self, batch: Dict):
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            trust_remote_code=True)

        template = "### Instruction:\n{}### Response:\n{}"
        texts = [template.format(inst, resp) for inst, resp in zip(batch["instruction"], batch["response"])]

        tokens = self.tokenizer(
            texts,
            padding='max_length',
            max_length=256,
            truncation=True,
            return_tensors='pt'
        )

        tokens['labels'] =  tokens['input_ids'].clone()
        return tokens

    def set_tokenized_data(self, data_type: str = 'train'):
        if data_type == "train":
            self.train_tokenized_data = self.train_dataset.map(self.preprocess_data, batched=True, remove_columns=self.train_dataset.column_names)
            return self.train_tokenized_data
        elif data_type == "validation":
            self.validation_tokenized_data = self.train_dataset.map(self.preprocess_data, batched=True, remove_columns=self.validation_dataset.column_names)
            return self.validation_tokenized_data
        elif data_type == "test":
            self.test_tokenized_data = self.train_dataset.map(self.preprocess_data, batched=True, remove_columns=self.test_dataset.column_names)
            return self.test_tokenized_data

    def train(self, training_args: Optional[Dict]=None, adapted_model_path: str='./tinyllama-lora-ftuned-adapted'):
        if not training_args or training_args is None:
            training_args = {
                "output_dir":'./tinyllama-lora-ftuned-ml',
                'per_device_train_batch_size':4,
                'gradient_accumulation_steps':4,
                'learning_rate':1e-3,
                'num_train_epochs':50,
                'fp16':True,
                'report_to':'none',
                'logging_steps':20,
                'save_strategy':'epoch',
                'eval_strategy': 'epoch',
                'save_total_limit': 3,
                'load_best_model_at_end': True,
                'metric_for_best_model': 'eval_loss',
                'greater_is_better': False,
                'remove_unused_columns': False,
                'label_names': ['labels']
            }

        self.training_args = TrainingArguments(**training_args)

        early_stopping = EarlyStoppingCallback(
            early_stopping_patience=3,
            early_stopping_threshold=0.01
        )

        trainer = Trainer(
            model = self.model,
            args = self.training_args,
            train_dataset = self.train_tokenized_data,
            eval_dataset = self.validation_tokenized_data,
            processing_class = self.tokenizer,
            callbacks=[early_stopping]
        )

        self.trainer = trainer

        print("Training the model")
        trainer.train()

        print("Saving the new weights")
        self.model.save_pretrained(adapted_model_path)
        self.tokenizer.save_pretrained(adapted_model_path)

    def evaluate_on_test(self):
        if not hasattr(self, "trainer"):
            raise RuntimeError("Trainer not found. Train the model before evaluation.")

        if self.test_tokenized_data is None:
            self.set_tokenized_data("test")

        print("Evaluating on test dataset...")
        results = self.trainer.evaluate(self.test_tokenized_data)

        # Perplexity is a common metric for language models
        import math
        try:
            results["perplexity"] = math.exp(results["eval_loss"])
        except OverflowError:
            results["perplexity"] = float("inf")

        print("Test Results:", results)
        return results


    def prepare_and_load_data(self):
            preprocessed_data = []
            data = None

            try:
                with open(self.data_path) as f:
                    data = json.load(f)
            except FileNotFoundError:
                print(f"Error: File not found -> {self.data_path}")
                return []
            except json.JSONDecodeError as e:
                print(f"Error: Invalid JSON format in {self.data_path} - {e}")
            except Exception as e:
                print(f"Unexpected error while loading file: {e}")
                return []

            questions = data.get("questions", [])
            if not isinstance(questions, list):
                print("Error: 'questions' should be a list in the dataset")
                return []

            for idx, q in enumerate(questions, start=1):
                try:
                    preprocessed_data.append({
                        "instruction": f"[{q['category']} - {q['difficulty']}] {q['instruction']}",
                        "input": "",
                        "response": q["response"]
                    })
                except KeyError as e:
                    print(f"Skipping question {idx}: Missing key {e}")
                except Exception as e:
                    print(f"Skipping question {idx}: due to unexpected error - {e}")

            with open("lora_dataset.json", "w") as f:
                json.dump(preprocessed_data, f, indent=2)

            dataset = load_dataset("json", data_files="lora_dataset.json")


            dataset = dataset['train'].train_test_split(test_size=0.3, seed=42)
            train_dataset = dataset['train']
            temp_dataset = dataset['test']

            temp_split = temp_dataset.train_test_split(test_size=0.5, seed=42)
            validation_dataset = temp_split['train']  # 15% of total
            test_dataset = temp_split['test']         # 15% of total

            self.train_dataset = train_dataset        # 70%
            self.validation_dataset = validation_dataset  # 15%
            self.test_dataset = test_dataset          # 15%

            print(f"Train samples: {len(self.train_dataset)}")
            print(f"Validation samples: {len(self.validation_dataset)}")
            print(f"Test samples: {len(self.test_dataset)}")

            return self.train_dataset, self.validation_dataset, self.test_dataset

In [14]:
data_path = "/content/drive/MyDrive/interview-questions/ml_interview_dataset.json"
model_name = 'TinyLLama/TinyLlama-1.1B-Chat-v1.0'

In [15]:
lora_trainer_obj = LoRATrainer(data_path=data_path, model_name=model_name)

In [16]:
lora_trainer_obj.prepare_and_load_data()

Generating train split: 0 examples [00:00, ? examples/s]

Train samples: 1843
Validation samples: 395
Test samples: 395


(Dataset({
     features: ['instruction', 'input', 'response'],
     num_rows: 1843
 }),
 Dataset({
     features: ['instruction', 'input', 'response'],
     num_rows: 395
 }),
 Dataset({
     features: ['instruction', 'input', 'response'],
     num_rows: 395
 }))

In [17]:
lora_trainer_obj.set_lora_conf()

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules={'v_proj', 'q_proj'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None)

In [18]:
lora_trainer_obj.set_bnb_config()

BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "bfloat16",
  "bnb_4bit_quant_storage": "uint8",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": false,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}

In [19]:
lora_trainer_obj.load_model()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear

In [20]:
lora_trainer_obj.set_tokenized_data("train")
lora_trainer_obj.set_tokenized_data("validation")

Map:   0%|          | 0/1843 [00:00<?, ? examples/s]

Map:   0%|          | 0/1843 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1843
})

In [21]:
lora_trainer_obj.train()

Training the model


Epoch,Training Loss,Validation Loss
1,0.158900,0.150224
2,0.155900,0.129041
3,0.125500,0.113787
4,0.125400,0.100278
5,0.091500,0.090946
6,0.095700,0.079601
7,0.076000,0.072021
8,0.080600,0.068708
9,0.071500,0.058931


Saving the new weights


In [22]:
test_results = lora_trainer_obj.evaluate_on_test()

Parameter 'function'=<bound method LoRATrainer.preprocess_data of <__main__.LoRATrainer object at 0x7ad76e3bf6b0>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1843 [00:00<?, ? examples/s]

Evaluating on test dataset...


Test Results: {'eval_loss': 0.058930832892656326, 'eval_runtime': 87.3934, 'eval_samples_per_second': 21.089, 'eval_steps_per_second': 2.643, 'epoch': 9.0, 'perplexity': 1.060701872522578}
